In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy import signal
import sys

In [2]:
circ_parameters_path = glob.glob('../dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

In [3]:
direc = r"../dataFolders/PaperPipelineOutput/v3/RawTracks/"
direc1 = r"../dataFolders/PaperPipelineOutput/v3/FilteredTracks/"
direc2 = r"../dataFolders/PaperPipelineOutput/v3/FilteredTracks/ManuallyTracked/"
outdirec = r"../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/"
visitnum = ['FirstVisit/', 'Later7thVisit/','Later20thVisit/', 'LastVisit/']

videoAnnotation = "AllVideoNames*.csv"
frameRef = "frameNumberReference.csv"

In [4]:
for visit in visitnum:
    outpath = os.path.join('../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/', visit)
    print(outpath)
    if not os.path.exists(outpath):
        try:
            os.mkdir(outpath)
        except OSError:
            print ("Creation of the directory %s failed" % outpath)

../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/FirstVisit/
../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/Later7thVisit/
../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/Later20thVisit/
../dataFolders/PaperPipelineOutput/v3/FilteredTracks/FinalCompiledTracks/LastVisit/


In [5]:
for visit in visitnum:
    path = os.path.join(direc1, visit)
    trackslist = glob.glob(path + 'c*.csv')
    
    f = glob.glob(os.path.join(direc2, visit) + videoAnnotation)
    VidAnnotation = pd.read_csv(f[0])
        
    automated = VidAnnotation.loc[VidAnnotation.AutomatedTracking == 'TRUE', 'names'].values
    manually = VidAnnotation.loc[VidAnnotation.Manual == 'Done', 'names'].values
    frameRead = pd.read_csv(glob.glob(os.path.join(direc2, visit) + frameRef)[0])
    
    for data in trackslist:
        name = os.path.basename(data)
        name = name.split('_')[0] + "_" + name.split('_')[1]
        
        df = pd.read_csv(data)
        
        if name in automated:  
            df.to_csv(outdirec + visit + name + '_' + visit[:-1] + '.csv')
        elif name in manually:
            newpath = os.path.join(direc2, visit)
            newlist = glob.glob(newpath + '*/' + name + '_*xypts.csv', recursive=True)[0]
            df = pd.read_csv(newlist)
            
            strt = frameRead[frameRead.mothID == name].startFrame.values[0]
            stp = frameRead[frameRead.mothID == name].StopFrame.values[0]
            
            df = df[strt:stp].reset_index(drop = True)
            df.columns = ['x', 'y']
            df.to_csv(outdirec + visit + name + '_' + visit[:-1] + '.csv')